In [2]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil
import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import src.aformer_atac_cage as aformer
#import src.aformer_TF as aformer
from src.layers.layers import *
import src.metrics as metrics
from src.optimizers import *
import src.schedulers as schedulers
import src.utils as utils

import training_utils_atac_cage as training_utils


from scipy import stats

In [3]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-5')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    #options.experimental_threading.max_intra_op_parallelism = 1
    mixed_precision.set_global_policy('mixed_bfloat16')
    tf.config.optimizer.set_jit(True)
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS

2023-02-25 16:16:50.196138: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-25 16:16:50.196211: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-25 16:16:50.196254: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-02-25 16:16:50.503944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 16:16:50.611735: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:42303
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-5


INFO:tensorflow:Initializing the TPU system: node-5


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
g = tf.random.Generator.from_seed(datetime.now().timestamp())
with strategy.scope():
    
    data_tr,data_val,data_val_TSS = training_utils.return_distributed_iterators("gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv/preprocessed",
                                                                                "gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_TSS/preprocessed",
                                                                               GLOBAL_BATCH_SIZE,
                                                                               196608,
                                                                               10, 
                                                                               1536,
                                                                               320,
                                                                               128,
                                                                               8,
                                                                               50,
                                                                               strategy,
                                                                               options,
                                                                                True,
                                                                                0.00,
                                                                               g)

gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv/preprocessed
gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_TSS/preprocessed
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


created train + val


In [5]:
with strategy.scope():
    inits = training_utils.get_initializers_enformer_performer("gs://picard-testing-176520/enformer_performer_pretrain_atac_mean/models/enformer_performer_pretrain_atac_mean_EP_full_dataset__196k_load_init-True_freeze-False_LR1-1e-06_LR2-0.0001_T-4_F-1536_D-0.4_2023-02-24_16:27:50/iteration_16",
                                                               4,
                                                               stable_variant=False)
    #inits = training_utils.get_initializers_enformer_conv("/home/jupyter/dev/BE_CD69_paper_2022/enformer_fine_tuning/checkpoint/sonnet_weights")
    model = aformer.aformer(kernel_transformation = 'relu_kernel_transformation',
                            dropout_rate = 0.05,
                            pointwise_dropout_rate=0.05,
                            input_length = 196608,
                            output_length = 1536,
                            final_output_length = 896,
                             num_heads = 8,
                             numerical_stabilizer = 0.001,
                             nb_random_features = 256,
                             hidden_size = 1552,
                             num_transformer_layers = 4,
                             d_model = 1552,
                             norm=True,
                             dim = 194, 
                            BN_momentum=0.90,
                             max_seq_length = 1536,
                             use_rot_emb = True,
                            fc_dropout=0.25,
                             use_mask_pos = False, 
                             normalize = True,
                             seed = 3,
                             load_init=True,
                            stable_variant=False,
                             inits=inits,
                            freeze_conv_layers=False,
                            predict_masked_atac_bool=True,
                            use_global_acc=False,
                            inits_type='enformer_performer')

True


In [6]:
with strategy.scope():
    optimizer1 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-06,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-08,
        rectify=True
    )
    optimizer2 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04,
        epsilon= 1.0e-14,
        weight_decay= 1.0e-0,
        rectify=True
    )
    optimizer3 = tfa.optimizers.AdaBelief(
        learning_rate= 1.0e-04,
        epsilon= 1.0e-14,
        weight_decay= 0.0,
        rectify=True
    )
    
    optimizers_in=optimizer1,optimizer2,optimizer3
    

In [7]:
with strategy.scope():
    metric_dict = {}
    train_step_masked_atac,train_step, \
        val_step_masked_atac,val_step, \
            val_step_TSS_masked_atac, val_step_TSS, \
                build_step, metric_dict = training_utils.return_train_val_functions(model,
                                                                                     500,
                                                                                     150,
                                                                                     150,
                                                                                      optimizers_in,
                                                                                      strategy,
                                                                                      metric_dict,
                                                                                      GLOBAL_BATCH_SIZE,
                                                                                        5.0,
                                                                                      0.8,
                                                                                    'poisson')
    


In [ ]:
from scipy.stats.stats import pearsonr  
with strategy.scope():
    def sum_log(x):
        return np.log10(1.0 + np.nansum(x))
    
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 10):
        start = time.time()
        if epoch_i == 1:
            build_step(data_val)
            total_params = 0
            for k in model.trainable_variables:
                var = k.values[0]
                total_params += tf.size(var)
            print('total params: ' + str(total_params))

        train_step_masked_atac(data_tr)

        print('hg_train_loss: ' + str(metric_dict['train_loss'].result().numpy()))
        
        val_step_masked_atac(data_val)

        val_losses.append(metric_dict['val_loss'].result().numpy())
        print('hg_val_loss: ' + str(metric_dict['val_loss'].result().numpy()))
        print('hg_val_loss_CAGE: ' + str(metric_dict['val_loss_CAGE'].result().numpy()))
        print('hg_val_loss_ATAC: ' + str(metric_dict['val_loss_ATAC'].result().numpy()))
        print('hg_val_cage_pearson: ' + str(metric_dict['CAGE_PearsonR'].result()['PearsonR'].numpy()))
        print('hg_val_cage_R2: ' + str(metric_dict['CAGE_R2'].result()['R2'].numpy()))
        
        #print('hg_val_atac_pearson: ' + str(metric_dict['ATAC_PearsonR'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_R2: ' + str(metric_dict['ATAC_R2'].result()['R2'].numpy()))
        
        
        #print('hg_val_atac_baseline_pearson: ' + str(metric_dict['ATAC_PearsonR_baseline'].result()['PearsonR'].numpy()))
        #print('hg_val_atac_baseline_R2: ' + str(metric_dict['ATAC_R2_baseline'].result()['R2'].numpy()))
        
        val_step_TSS_masked_atac(data_val_TSS)
        
        y_trues = metric_dict['corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['corr_stats'].result()['gene_map'].numpy()


        figures, corrs_overall = training_utils.make_plots(y_trues,
                                           y_preds, 
                                           cell_types, 
                                           gene_map)
        
        cell_spec,gene_spec = corrs_overall
        
        print('cell spec corr: ' + str(cell_spec))
        print('gene_spec_corr:' + str(gene_spec))
        

        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))
        
        for key, item in metric_dict.items():
            item.reset_state()
        

total params: tf.Tensor(132783826, shape=(), dtype=int32)
hg_train_loss: 0.0034167897
hg_val_loss: 0.012973947
hg_val_loss_CAGE: 0.0040297396
hg_val_loss_ATAC: 0.00894421
hg_val_cage_pearson: [0.33343163]
hg_val_cage_R2: [0.0725764]
cell spec corr: 0.6574068744900461
gene_spec_corr:0.40471770426476605
completed epoch 1
duration(mins): 12.527095707257589
patience counter at: 0


In [ ]:
model.stem_res_conv._layer

In [24]:
model.final_pointwise_conv.layers[0].momentum

0.9

In [25]:
model.final_pointwise_conv.layers[0].momentum = 0.70

In [26]:
model.final_pointwise_conv.layers[0].momentum

0.7

In [44]:
def deserialize_tr(serialized_example,
                   input_length,
                   max_shift,
                   output_length,
                   crop_size,
                   output_res,
                   predict_masked_atac_bool,
                   atac_mask_dropout,
                   g):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'atac': tf.io.FixedLenFeature([], tf.string),
        'cage': tf.io.FixedLenFeature([], tf.string),
        'processed_gene_token': tf.io.FixedLenFeature([], tf.string)
    }
    ### stochastic sequence shift and gaussian noise


    rev_comp = tf.math.round(g.uniform([], 0, 1))

    shift = g.uniform(shape=(),
                      minval=0,
                      maxval=max_shift,
                      dtype=tf.int32)

    for k in range(max_shift):
        if k == shift:
            interval_end = input_length + k
            seq_shift = k
        else:
            seq_shift=0
    
    input_seq_length = input_length + max_shift


    data = tf.io.parse_example(serialized_example, feature_map)
    #sequence = one_hot(tf.strings.substr(data['sequence'],
    #                             seq_shift,input_length))
    
    atac = tf.ensure_shape(tf.io.parse_tensor(data['atac'],
                                              out_type=tf.float32),
                           [output_length,1])
    #atac = atac + tf.math.abs(g.normal(atac.shape,
    #                                           mean=0.0,
    #                                           stddev=0.05,
    #                                           dtype=tf.float32))
    
    masked_atac=tf.nn.experimental.stateless_dropout(atac, 
                                                     rate=atac_mask_dropout, 
                                                     seed=[0,seq_shift]) / (1. / (1.0 -atac_mask_dropout))
    
    cage = tf.ensure_shape(tf.io.parse_tensor(data['cage'],
                                              out_type=tf.float32),
                           [output_length - 2*crop_size,1])

        
    gene_token= tf.io.parse_tensor(data['processed_gene_token'],
                                   out_type=tf.int32)
        
    if predict_masked_atac_bool:
        atac_out = tf.slice(atac,
                            [crop_size,0],
                            [output_length-2*crop_size,-1])
        target = tf.concat([atac_out,cage],axis=1)
        
    if predict_masked_atac_bool:
    
        return {#'sequence': tf.ensure_shape(sequence,
                #                            [input_length,4]),
                'atac': tf.ensure_shape(masked_atac,
                                          [output_length,1]),
                'gene_token':gene_token,
                'target': tf.ensure_shape(target,
                                          [output_length-crop_size*2,2])}
    else:
        return {#'sequence': tf.ensure_shape(sequence,
                #                            [input_length,4]),
                'atac': tf.ensure_shape(atac,
                                          [output_length,1]),
                'gene_token':gene_token}
                #'target': tf.ensure_shape(cage,
                #                          [output_length-crop_size*2,1])}
    
    
g = tf.random.Generator.from_seed(datetime.now().timestamp())
dataset = tf.data.TFRecordDataset("gs://picard-testing-176520/genformer_atac_rampage_globalacc_conv_TSS/valid/HG_A375.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)

dataset = dataset.map(lambda record: deserialize_tr(record,
                                                    196618,
                                                    10,
                                                    1536,
                                                    320,
                                                    128,
                                                    False,
                                                    0.0,
                                                    g),
                      deterministic=True,
                      num_parallel_calls=4)
        
    
dataset1 = tf.data.TFRecordDataset("gs://picard-testing-176520/enformer_atac_rampage_paired_basenji_tss/valid/HG_A375.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)

dataset1 = dataset1.map(lambda record: deserialize_tr(record,
                                                    196618,
                                                    10,
                                                    1536,
                                                    320,
                                                    128,
                                                    False,
                                                    0.0,
                                                    g),
                      deterministic=True,
                      num_parallel_calls=4)
        

In [49]:
test = iter(dataset)

In [55]:
next(test)

{'atac': <tf.Tensor: shape=(1536, 1), dtype=float32, numpy=
 array([[0.45809558],
        [0.47703928],
        [1.2499237 ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]], dtype=float32)>,
 'gene_token': <tf.Tensor: shape=(), dtype=int32, numpy=6870>}

In [51]:
test = iter(dataset1)

In [56]:
next(test)

{'atac': <tf.Tensor: shape=(1536, 1), dtype=float32, numpy=
 array([[0.        ],
        [0.        ],
        [0.72191423],
        ...,
        [5.7844844 ],
        [1.4600831 ],
        [0.        ]], dtype=float32)>,
 'gene_token': <tf.Tensor: shape=(), dtype=int32, numpy=15816>}